In [ ]:
import pandas as pd

df = pd.read_csv('option-chain-ED-NIFTY-25-Jul-2024.csv', skiprows = 1)
df

,Unnamed: 0,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,...,BID.1,ASK.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,CHNG IN OI.1,OI.1,Unnamed: 22
0,NaN,536,-,-,-,"3,922.15",38.50,525,"3,972.25","4,179.40",...,4.95,5.20,"1,800",-0.85,4.95,30.66,544,209,"1,001",NaN
1,NaN,"2,131",-13,21,-,"3,940.00",39.10,525,"3,940.25","4,087.65",...,4.90,5.70,900,-0.15,5.15,30.45,"1,595",185,"7,644",NaN
2,NaN,218,-,-,-,"3,537.70",-0.30,525,"3,635.25","4,076.85",...,5.05,5.95,125,-1.80,5.05,30.01,10,10,90,NaN
3,NaN,307,20,20,-,"3,830.00",-128.85,525,"3,840.60","4,008.40",...,4.70,8.30,175,-0.50,5.60,30.04,66,-7,275,NaN
4,NaN,206,-,-,-,"3,720.00",-,525,"3,763.60","3,981.05",...,4.10,-,-,-,7.00,-,-,-,78,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,NaN,"2,975","2,975","6,588",11.29,10.30,-39.70,325,7.70,10.60,...,"1,328.00","1,424.40",450,-29.00,"1,371.00",18.48,15,15,40,NaN
110,NaN,868,89,478,11.31,8.75,-1.85,250,8.55,10.75,...,"1,377.05","1,602.75",525,-,-,-,-,-,-,NaN
111,NaN,"1,007",244,"1,520",11.48,8.05,-1.35,125,8.10,10.95,...,"1,467.80","1,526.30",450,"-1,231.70","1,478.00",19.96,3,3,3,NaN
112,NaN,81,81,226,12.36,10.50,-31.00,100,8.60,10.80,...,"1,466.30","1,704.35",525,-,-,-,-,-,-,NaN


In [ ]:
chain = df[["LTP", "STRIKE", "LTP.1"]].copy()
chain.rename(columns = {"LTP": "C", "STRIKE": "K", "LTP.1": "P"}, inplace = True)
chain

,C,K,P
0,"3,922.15","20,150.00",4.95
1,"3,940.00","20,200.00",5.15
2,"3,537.70","20,250.00",5.05
3,"3,830.00","20,300.00",5.60
4,"3,720.00","20,350.00",7.00
...,...,...,...
109,10.30,"25,600.00","1,371.00"
110,8.75,"25,650.00",-
111,8.05,"25,700.00","1,478.00"
112,10.50,"25,750.00",-


In [ ]:
S = 24141.95
r = 0.065
t = "01-07-2024"
T = "25-07-2024"

from datetime import datetime

def interval(date1, date2):
    date_format = "%d-%m-%Y"
    d1 = datetime.strptime(date1, date_format)
    d2 = datetime.strptime(date2, date_format)
    difference_in_days = abs((d2 - d1).days)
    years = difference_in_days/365.0
    return years

time = interval(t,T)

import numpy as np

def clean_and_convert(column):
    if chain[column].dtype == 'object':
        chain[column] = chain[column].str.replace(",", "", regex=True)
    chain[column] = pd.to_numeric(chain[column], errors="coerce")

for column in ["C", "K", "P"]:
    clean_and_convert(column)

chain = chain.dropna()

chain.loc[:, "K_discounted"] = chain["K"]*np.exp(-r*time)
chain.loc[:, "Deviation"] = chain["C"] + chain["K_discounted"] - chain["P"] - S
chain.loc[:, "Arbitrage"] = np.where(
    chain["Deviation"] > 0,
    chain["Deviation"],
    -chain["Deviation"],
)

arbitrage_rows = chain[chain["Deviation"] != 0]
total_profit = arbitrage_rows["Arbitrage"].sum()

print("Arbitrage Opportunities:")
print(arbitrage_rows)
print(f"Total Arbitrage Profit: ₹{total_profit:.2f}")

Arbitrage Opportunities:
           C        K        P  K_discounted   Deviation   Arbitrage
0    3922.15  20150.0     4.95  20064.063229 -160.686771  160.686771
1    3940.00  20200.0     5.15  20113.849986  -93.250014   93.250014
2    3537.70  20250.0     5.05  20163.636743 -445.663257  445.663257
3    3830.00  20300.0     5.60  20213.423501 -104.126499  104.126499
4    3720.00  20350.0     7.00  20263.210258 -165.739742  165.739742
..       ...      ...      ...           ...         ...         ...
107    12.35  25500.0  1286.00  25391.246269  -24.353731   24.353731
108    14.95  25550.0  1507.35  25441.033027 -193.316973  193.316973
109    10.30  25600.0  1371.00  25490.819784  -11.830216   11.830216
111     8.05  25700.0  1478.00  25590.393299  -21.506701   21.506701
113     7.80  25800.0  1600.00  25689.966814  -44.183186   44.183186

[110 rows x 6 columns]
Total Arbitrage Profit: ₹11874.59
